DB给我的感觉是面试官心里有一个想要的答案，但他会期待你自己drive the conversation, 主动把可能存在的问题和解决方案一并提出来。我还是面E5级别的，E6的话不知道标准如何。你面试完有没有找朋友复盘一下看看哪里可能有问题呢

# Delete index from interval
Array of intervals。给一个interval array,每个interval不相交。再给一个所有interval cover的list中的index。要求给出删掉这个index所cover的element之后的interval array.
比如：
有这样一个array: [(4, 7), (10, 11), (13, 15)]，它所cover的list是: [4, 5, 6, 7, 10, 11, 13, 14, 15]。如果index是2的话，把6从covered list中删掉，output是[(4, 5), (7,7), (10, 11), (13, 15)]。

我的解法是根据prefix sum算出这个index落在哪个interval里，然后divide这个interval。但是得到的feedback是overly complicated。也不知道有什么更好的解法。还有这题有很多的edge case要考虑到。这轮坑的地方在于我写好base solution正要检查呢面试官就着急忙慌地说有个edge case没考虑到。等我fix了之后马上说还有这个edge case没考虑。真是一点让我自己debug的机会都不给。最后给的feedback是需要explicitly告诉我edge case在哪里。感觉他们就是希望能够写的时候一遍过，which我觉得在现实开发中一点也不现实啊。

关于coding的部分那个关于Interval的问题，我的思路跟楼主查不过，每个interval的长度是end - start + 1，所以直接遍历一遍，如果要删的index还没到（或者已经超过了）就直接输出到output，如果index在当前的interval里面，就把这个interval变成两个加回去。具体code可以看下面。请问这样是overly complicated了么？我想不到更简单的了

vector<vector<int>> deleteIndex(const vector<vector<int>>& intervals, int index) {
  // 如果intervals没sort的话，需要sort一下
  vector<vector<int>> res;
  for (auto& interval : intervals) {
    if (index >= 0 && index <= (interval[1] - interval[0])) {
      if (index != 0)  res.push_back({interval[0], index + interval[0] - 1});
      if (index != interval[1] - interval[0]) res.push_back({index + interval[0] + 1, interval[1]});
    } else {
      res.push_back(interval);
    }
    index -= interval[1] - interval[0] + 1;
  }
  return res;
}

interval 的 coding 题，如果 array 是 sorted 的话，是不是可以先用二分法找到是否有一个 interval 包含了 index，如果包含了，把这个 interval 拆分，其余的按原来的保留


In [ ]:
"""
1. is the array of intervals sorted?
2. does the interval have overlapping or not?

（4，7）
4，5，6，7 for the covered list
index is 0,1,2,3 here
delete_index = 5, not in the first interval

[10, 11]
index 0, 1
delete_index - length_of_prev_interval = 5-4=1
so we need to find the 1st index in second interval, which is 11.

we have [10,10] left.


"""
def delete_index(intervals, index):
    # assume interval are sorted and no overlapping
    res = []
    for start, end in intervals:
        # 计算每个interval的长度
        interval_length = end-start+1

        # if the index falls inside the interval
        if 0<=index<interval_length:
            value_to_delete = start + index 

            # if the delete value is not the start, then we can keep start to val-1
            if value_to_delete > start:
                res.append([start, value_to_delete-1])
            
            # if the delete value is not the end, then we can keep val+1 to end
            if value_to_delete < end:
                res.append([value_to_delete+1, end])
            
            # after we deal with the index, we can label it to non-operational index. All following intervals will be simply appended.
            index = -1
        
        # if index not inside the interval, index-interval_length给下一次做准备
        else:
            res.append([start, end])
            index -= interval_length
    return res 
delete_index([(4,7), (10, 11), (13, 15)], 5)

# time: O(n)
# space: O(1)




[[4, 7], [10, 10], [13, 15]]

In [13]:
# 如果需要重复删除，或者interval数量很多的情况下, 上述方法是O（n），不够efficient
# 用二分方法，先建立prefixSum，O（n），后面每次二分查找只需要O（logn），更加scalable
from bisect import bisect_right
"""
intervals = [(4,7), (10,11), (13,15)]
flattened list = [4,5,6,7,10,11,13,14,15]

construct a prefixSum array to represent the start location of each interval in the flattend list.append
prefix_sum = [0, 4, 6, 9]
[0,4) -> 1st interval
[4,6) -> 2nd interval
[6,9) -> 3rd interval
"""
def build_prefix_sum(intervals):
    prefix_sum = [0]
    for start, end in intervals:
        interval_length = end-start+1
        prefix_sum.append(prefix_sum[-1] + interval_length)
    return prefix_sum

def binary_search(arr, i):
    l, r = -1, len(arr)
    while l+1 != r:
        mid = (r-l)//2+l
        if arr[mid] < i:
            l = mid
        else:
            r = mid 
    return r

def delete_index(intervals, index): # intervals，6
    prefix_sum = build_prefix_sum(intervals)

    # find the ith interval that the index belongs to 
    
    i = binary_search(prefix_sum, index) # 2
    print(i)

    res = []

    for j, (start, end) in enumerate(intervals):
        if j != i:
            res.append([start, end])
        else:
            # local_index 是当前interval内的偏移位置
            # index是global index，减去prefix sum中的累积的index数量；
            # 
            local_index = index - prefix_sum[i] # local_index = 6-6=0
            value_to_delete = start + local_index # value_to_delete = 13+0=13, so we want to delete val 13.

            # 然后检测这个13 是否在两个端点，如果不在，就可以拆成两部分结果里面，如果在，就只能加一部分
            if value_to_delete > start:
                res.append([start, value_to_delete-1])
            if value_to_delete < end:
                res.append([value_to_delete+1, end])
    return res
intervals = [(4,7), (10,11), (13,15)]
# prefix = [0, 4, 6, 9]
delete_index(intervals, 6)

# prefix_sum can be cached, to consistently delete, O(logn) time for each deletion.

2


[[4, 7], [10, 11], [14, 15]]

# Revenue and Referral
Design Revenue System API

三种操作

1. insert(revenue): 返回一个auto-increment的customer id
2. insert(revenue, referrer) ：返回一个auto-increment的customer id，和1的不同是这个新的customer是被referrer refer的
3. get_top_k_revenue(k, min_revenue) -> set[int]：返回有topk revenue的customer，但是要满足revenue 不小于min_revenue。每个customer的revenue是自己的revenue和被他直接refer的customer的revenue总和。

每个user只会被插入一次revenue（不可以重复插入或者更新）
然后在他们refer别人，别人加入的时候，也要更新这个user的revenue

还要更新每个人的所有的revenue

In [ ]:
# Linear scan
# Binary search
import heapq

class RevenueAPI:
    def __init__(self):
        self.next_id = 1
        self.revenue = {} # id->own revenue
        self.referrals = {} # user id who are referring->list of referred ids
        self.total_revenue = {} # id->own+direct referrals revenue

    
    def insert(self, revenue):
        cid = self.next_id
        self.next_id += 1

        self.revenue[cid] = revenue
        self.referrals[cid] = []
        self.total_revenue[cid] = revenue

        return cid
    
    def insert_with_referral(self, revenue, referrer):
        cid = self.insert(revenue)
        self.referrals[referrer].append(cid)

        self.total_revenue[referrer] += revenue 

        return cid
    
    def get_top_k_revenue(self, k, min_revenue):
        heap = []
        for cid, total in self.total_revenue.items():
            if total >= min_revenue:
                heapq.heappush(heap, (total, cid))
                if len(heap) > k:
                    heapq.heappop(heap)
        
        return set(cid for total, cid in heap)

sys = RevenueAPI()

a = sys.insert(100)                 # id = 1
b = sys.insert_with_referral(50, a)  # id = 2, referred by 1
c = sys.insert(70)                 # id = 3
d = sys.insert_with_referral(90, c)  # id = 4, referred by 3
e = sys.insert(30)                 # id = 5

print(sys.get_top_k_revenue(2, 100))  # 应该返回 {1, 3}

# time: top_k, O(nlogk), frequent insert, but fewer top-k query, very suitable

# 如果插入少，查询多； 我们希望查询更加efficient，我想到的方法有 SortedList或者TreeMap，可以保证O（k）的查询和O（logn）的更新

# SortedList是一种自动保持元素有序的数据结构（底层是由balanced binary search tree实现的），认为是一个在插入或者删除的时候自动维护排序的list，
# Treemap是Java C++中基于balanced BST，（redblack tree or AVL tree）的map，key是有序的，插入删除查找都是logn
# 


{1, 3}


# Fibonacci tree move sequence
已知一个fibo tree，给出两个节点，找出连接两个节点的路径
Fibonacci tree

https://stackoverflow.com/questions/76142856/finding-a-path-between-two-nodes-in-a-k-th-order-fibonacci-tree

In [ ]:
def getDirections(root, startValue, destValue):
    # key is to find the LCA
    # the route must pass LCA, LCA to start and LCA to end to get the two strings. 
    # the string_lca_to_start will all be reverted to U, then add the string_lca_to_end to the below
    def find_lca(node, start, dest):
        # lca
        if not node or node.val == start or node.val == dest:
            return node 
        left = find_lca(node.left, start, dest) # 在左子树找到start or end
        right = find_lca(node.right, start, dest) # 在右子树找到start or end

        if left and right: # 两边各自找到一个，说明当前node就是lca
            return node
        return left or right # 否则，说明lca在一边，返回不是空的那边

    def find_path(node, val, path):
        # DFS + backtracking

        # 当前节点为空，说明走到头，返回False，说明没找到目标节点
        if not node:
            return False
        
        # 找到了val，返回True
        if node.val == val:
            return True
        
        # 如果节点不为空并且没找到val，我们开始尝试走左子树，如果找到了目标，返回True，否则 撤销上一步
        path.append('L')
        if find_path(node.left, val, path):
            return True
        path.pop()

        # 同理，走右子树，找到了返回True，否则撤销
        path.append('R')
        if find_path(node.right, val, path):
            return True
        path.pop()
        
        # 如果没有找到目标节点，就返回False
        return False
    
    lca = find_lca(root, startValue, destValue)
    pathToStart, pathToEnd = [], []
    find_path(root, startValue, pathToStart)
    find_path(root, destValue, pathToEnd)

    return 'U'*len(pathToStart)+''.join(pathToEnd)









In [17]:
# fib = [0,1]
# for i in range(2, 50):
#     fib.append(fib[i-1]+fib[i-2])

# def get_path(order, id, path=""):
#     if order <= 1:
#         return path 
#     left_size = fit[order]-1
#     if id == 0:
#         return path
#     elif id <= left_size:
#         return get_path(order-2, id-1, path+'L')
#     else:
#         return get_path(order-1, id-left_size-1, path+'R')
    


# def get_directions(order, src, dst):
#     def build_path(order, id):
#         path = []
#         node = id
#         while node != 0:
#             parent, move = find_parent(order, node)
#             path.append(move)
#             node = parent
#         return path[::-1] # from root to node

#     def find_parent(order, id):
#         left_size = fib[order]-1
#         if id <= left_size:
#             parent = 0
#             move = "L"
#             sub_parent, sub_move = find_parent(order-2, id-1)
#             return (sub_parent+1, move+sub_move)
#         else:
#             parent = 0
#             move = 'R'
#             sub_parent, sub_move = find_parent(order-1, id-left_size-1)
#             return (sub_parent+left_size+1, move+sub_move)

#     path1 = build_path(order, src)
#     path2 = build_path(order, dst)

#     i = 0
#     while i<len(path1) and i<len(path2) and path1[i]==path2[i]:
#         i+=1
#     return 'U'*(len(path1)-i) + ''.join(path2[i:])


# print(get_directions(5, 5, 7))



# DB HQ to SF 起点到终点三个options/通勤方式
就是那道从起点到终点3个options 的题目， 一时大意了，发现其实不同option同时广搜还是有一些corner case需要处理的，结果勉强完成第一问，没时间去解决follow up 问题。 所以别看题目不难，还是得要把手练熟。

从起点 S 走到终点 D，你只能用一种交通工具（只能走相同数字的格子），你要找出走得最快的一种方式。如果有多个一样快，就选最省钱的那个方式

/*
Fastest Route to Databricks HQ
You live in San Francisco city and want to minimize your commute time to the Databricks HQ.
Given a 2D matrix of the San Francisco grid and the time as well as cost matrix of all the available transportation
modes, return the fastest mode of transportation. If there are multiple such modes then return one with the least cost.
Rules:
1. The input grid represents the city blocks, so the commuter is only allowed to travel along the horizontal and vertical axes.
Diagonal traversal is not permitted.
2. The commuter can only move to the neighboring cells with the same transportation mode.
Sample Input:
2D Grid:              Legend:
|3|3|S|2|X|X|         X = Roadblock
|3|1|1|2|X|2|         S = Source
|3|1|1|2|2|2|         D = Destination
|3|1|1|1|D|3|         1 = Walk, 2 = Bike, 3 = Car, 4 = Train
|3|3|3|3|3|4|
|4|4|4|4|4|4|
Transportation Modes:        ["Walk", "Bike", "Car", "Train"]
Cost Matrix (Dollars/Block): [0,      1,      3,     2]
Time Matrix (Minutes/Block): [3,      2,      1,     1]
NOTE: In this example, we are only counting the blocks between 'S' and 'D' to compute the minimum time & dollar cost.
Sample Output: Bike
*/
/*
walk:  minTime, dollar ?
[0,2], [1,2], .... [3,4] 4 steps minTime: 4 * 3 = 12, cost: 4 * 0 = 0
bike: [0,2], [0,3], ... 4 stpes.  minTime: 4 * 2 = 8, cost: 4 * 1 =4
car:  ...
train: ...
for each transportation:
  bfs to find num of blocks
  if cannot reach to desti, ignore this type
  els:
    update minTime, minCost
o(m*n) time
o(m*n) space
   0 1 2 3 4 5
0 |3|3|S|2|X|X|
1 |3|1|1|2|X|2|
2 |3|1|1|2|2|2|
3 |3|1|1|1|D|3|
4 |3|3|3|3|3|4|
5 |4|4|4|4|4|4|
grid = {
  {'3', '3', 'S', '2', 'X', 'X'},
  {'3', '1', '1', '2', 'X', '2'},
  {'3', '1', '1', '2', '2', '2'},
  {'3', '1', '1', '1', 'D', '3'},
  {'3', '3', '3', '3', '3', '4'},
  {'4', '4', '4', '4', '4', '4'}
};
cost_matrix = {0, 1, 3, 2};
time_matrix = {3, 2, 1, 1};
*/

In [37]:
from collections import deque

def find_fastest_transportation(grid, time, cost):
    """
    1. Find the start and destination coordinates
    2. create BFS queue with start point, modality, time, cost; create visited set; init min time and cost for each transportion mode.
    3. using BFS to explore all paths
    """
    start, end = None, None 
    ROWS = len(grid)
    COLS = len(grid[0])
    dirs = [(1, 0), (-1, 0), (0, 1), (0, -1)]
    n = len(time) # number of modality


    for r in range(ROWS):
        for c in range(COLS):
            if grid[r][c] == 'S':
                start = (r, c)
            if grid[r][c] == 'D':
                end = (r, c)
    
    if not start or not end:
        return None
    
    best_time = float('inf')
    best_cost = float('inf')
    best_mode = None 

    for mode in range(1, n+1):
        q = deque()
        visited = set()
        sr, sc = start 

        q.append((sr, sc, 0)) # (r, c, steps)
        visited.add((sr, sc))
        found = False 

        while q:
            r, c, steps = q.popleft()
            for dr, dc in dirs:
                nr, nc = r+dr, c+dc 
                if 0<=nr<ROWS and 0<=nc<COLS and (nr, nc) not in visited:
                    curr = grid[nr][nc]
                    if curr == 'X':
                        continue 
                    if curr == 'D':
                        # found destination
                        steps += 1
                        total_time = steps * time[mode-1]
                        total_cost = steps * cost[mode-1]
                        if (total_time < best_time) or (total_time == best_time and total_cost < best_cost):
                            best_time = total_time
                            best_cost = total_cost 
                            best_mode = mode
                        found = True
                        break 
                    elif curr == str(mode):
                        visited.add((nr, nc))
                        q.append((nr, nc, steps+1))
            if found:
                break
    return best_mode, best_time, best_cost
    
grid = [
    ['3','3','S','2','X','X'],
    ['3','1','1','2','X','2'],
    ['3','1','1','2','2','2'],
    ['3','1','1','1','D','3'],
    ['3','3','3','3','3','4'],
    ['4','4','4','4','4','4']
]

cost_matrix = [0, 1, 3, 2]     # 1-Walk, 2-Bike, 3-Car, 4-Train
time_matrix = [3, 2, 1, 1]

print(find_fastest_transportation(grid, time_matrix, cost_matrix))  # Output: Bike


(2, 10, 5)


# LC 981 - Time Based KV Store


In [40]:
from collections import defaultdict
class TimeMap:

    def __init__(self):
        self.hashmap = defaultdict(list)
        

    def set(self, key: str, value: str, timestamp: int) -> None:
        self.hashmap[key].append([value, timestamp])
        

    def get(self, key: str, timestamp: int) -> str:
        if key not in self.hashmap:
            return ""
        A = self.hashmap[key]
        l, r = -1, len(A)

        while l+1 != r:
            mid = (r-l)//2+l
            if A[mid][1] <= timestamp:
                l = mid
            else:
                r = mid
        
        return A[l][0] if l!=-1 else ""
        


# Your TimeMap object will be instantiated and called as such:
# obj = TimeMap()
# obj.set(key,value,timestamp)
# param_2 = obj.get(key,timestamp)

"""
一些可能的follow up
1️⃣ 支持并发访问怎么办？（系统设计方向）

Q: 如果这个系统部署在生产环境，多个线程并发调用 set 和 get，你如何保证线程安全？

回答点：
	•	Python 中可考虑加锁 threading.Lock（或者在 C++/Java 中用 mutex）
	•	如果是部署在服务端，要么用线程安全的数据结构，要么做请求队列和异步处理
	•	更通用方法是 用数据库后端（如 Cassandra / DynamoDB），而不是仅依赖内存 hashmap

⸻
2️⃣ 如果 timestamp 不是递增的怎么办？

Q: 当前 set 默认假设是递增插入（如 1, 2, 3），但如果插入乱序怎么办？

你的代码用 append 方式建立时间序列，是依赖 timestamp 单调递增的。

应对策略：
	•	在插入时用 bisect.insort() 把 (timestamp, value) 插入保持排序（代价 O(n)）
	•	或者改用 SortedDict（来自 sortedcontainers），可以 O(log n) 插入和查找

3️⃣ 空间优化（稀疏场景）

Q: 如果 value 每次都重复（即 timestamp 之间值不变），你还要存吗？

策略：
	•	压缩：只记录当 value 改变的时刻，节省空间
	•	用 run-length encoding 的思路压缩：[("on", 5), ("off", 10)] 表示 5~10 是 “on”
4️⃣️⃣️⃣ get(key, start_ts, end_ts): 支持区间查询

Q: 现在业务需要支持某 key 在 [start, end] 之间的所有值变化，怎么改？

思路：
	•	还是用二分找到第一个 ≥ start_ts 的下标
	•	然后线性扫描直到 timestamp > end_ts

⸻

5️⃣ 如果数据量大（亿级），如何 scale？

Q: 如果 key 的数量上亿，且每个 key 都有大量 timestamps，怎么 scale？

可以讨论：
	•	用 LSM tree、RocksDB 这种 time-series friendly 的 key-value 存储
	•	数据 sharding，按 key 或时间 hash
	•	存入 cloud object store（S3）+ 元数据在内存中
"""

'\n一些可能的follow up\n1️⃣ 支持并发访问怎么办？（系统设计方向）\n\nQ: 如果这个系统部署在生产环境，多个线程并发调用 set 和 get，你如何保证线程安全？\n\n回答点：\n\t•\tPython 中可考虑加锁 threading.Lock（或者在 C++/Java 中用 mutex）\n\t•\t如果是部署在服务端，要么用线程安全的数据结构，要么做请求队列和异步处理\n\t•\t更通用方法是 用数据库后端（如 Cassandra / DynamoDB），而不是仅依赖内存 hashmap\n\n⸻\n2️⃣ 如果 timestamp 不是递增的怎么办？\n\nQ: 当前 set 默认假设是递增插入（如 1, 2, 3），但如果插入乱序怎么办？\n\n你的代码用 append 方式建立时间序列，是依赖 timestamp 单调递增的。\n\n应对策略：\n\t•\t在插入时用 bisect.insort() 把 (timestamp, value) 插入保持排序（代价 O(n)）\n\t•\t或者改用 SortedDict（来自 sortedcontainers），可以 O(log n) 插入和查找\n\n3️⃣ 空间优化（稀疏场景）\n\nQ: 如果 value 每次都重复（即 timestamp 之间值不变），你还要存吗？\n\n策略：\n\t•\t压缩：只记录当 value 改变的时刻，节省空间\n\t•\t用 run-length encoding 的思路压缩：[("on", 5), ("off", 10)] 表示 5~10 是 “on”\n4️⃣️⃣️⃣ get(key, start_ts, end_ts): 支持区间查询\n\nQ: 现在业务需要支持某 key 在 [start, end] 之间的所有值变化，怎么改？\n\n思路：\n\t•\t还是用二分找到第一个 ≥ start_ts 的下标\n\t•\t然后线性扫描直到 timestamp > end_ts\n\n⸻\n\n5️⃣ 如果数据量大（亿级），如何 scale？\n\nQ: 如果 key 的数量上亿，且每个 key 都有大量 timestamps，怎么 scale？\n\n可以讨论：\n\t•\t用 LSM tree、RocksDB 这种 time-series f

# LC1146 Snapshot Array

实现一个set，需要一个支持当前状态的iterator，无论是put还是remove，不影响iterator之后的遍历 - LC1146

https://stackoverflow.com/questions/75431526/create-a-space-efficient-snapshot-set


这样记录下来就发现最后每个iterator只需要遍历map里面的所有KV pair，找到V数组里面<=自己记录的version number的最后的操作，是add就证明有这个值，空或者是remove就证明没有这个值。

其实这个类似基本的数据库snapshot的实现方式version vector，有想法最重要，实现起来还挺简单的，真佩服这题出的确实挺精妙的！

In [47]:
from collections import defaultdict
import bisect
class SnapshotArray:
    """
    1. very naive approach
    every time take a snapshot, save it. total space time is O(nk)
    n is the length, k is the number of snapshot.

    2. a little better approach
    snap[index] = {{snap_id, val}}
    = {{0, 1}, {1, 2}, {2, 2}, {3, 4}, {4, 4}....}
    seems there are some duplication for {2,2}, {4,4}; if the value does not change, we don't need to store every time when we takes a snapshot.
    = {{0,1}, {1,2}, {3,4}, {5,3}}
    so it implies us to find the last version id that is <= the queried snap_id. -> binary search upper bound

    """

    def __init__(self, length: int):
        self.hashmap = defaultdict(list)
        self.version = 0
        for key in range(length):
            self.hashmap[key] = [[-1, 0]]
        
        

    def set(self, index: int, val: int) -> None:
        self.hashmap[index].append([self.version, val])

    def snap(self) -> int:
        self.version += 1
        return self.version-1

    def get(self, index: int, snap_id: int) -> int:
        versions = [version for version, val in self.hashmap[index]]
        # binary search to find the 1st version id that is larger than the snap_id.
        l, r = 0, len(versions)-1
        while l<=r:
            mid = (r-l)//2+l
            if versions[mid] <= snap_id:
                l = mid + 1
            else:
                r = mid - 1
        # if r==-1: return 0
        return self.hashmap[index][r][1]
# time: init, O(n), set and snap are O(1), get is O(log(n_version))

# Your SnapshotArray object will be instantiated and called as such:
# obj = SnapshotArray(length)
# obj.set(index,val)
# param_2 = obj.snap()
# param_3 = obj.get(index,snap_id)

# Tic tac toe, M x N 棋盘 - LC348
判断胜利是after each move，不需要考虑invalid case
board不是3x3，而是任意尺寸，而且获胜条件的连成一条线的棋子个数也是可以配置的。
LC348 - Design Tic-Tac-Toe

In [42]:
class TicTacToe:
    def __init__(self, m, n, k):
        """
        mxn case board
        k: the number of placement that will be successful
        """
        self.m = m
        self.n = n
        self.k = k 
        self.board = [[0] * n for _ in range(m)]

    def move(self, row, col, player):
        self.board[row][col] = player

        dirs = [(0,1), (1,0), (1,1), (-1,1)] # four directions, 左右走，上下走，diag，anti-diag

        for dr, dc in dirs:
            count = 1

            # forward,从棋子落点向前走
            r, c = row+dr, col+dc
            while 0<=r<self.m and 0<=c<self.n and self.board[r][c] == player:
                count += 1
                # 如果r c还是player走过的，那就接着往右边走，走到头或者不是这个player的棋子
                r += dr 
                c += dc 
            
            # backward，从棋子落点向后走
            r, c = row-dr, col-dc 
            while 0<=r<self.m and 0<=c<self.n and self.board[r][c] == player:
                count += 1
                r -= dr
                c -= dc 
            

            if count >= self.k:
                return player 
        
        return 0






game = TicTacToe(3, 3, 3)
assert game.move(0, 0, 1) == 0
assert game.move(0, 1, 2) == 0
assert game.move(1, 1, 1) == 0
assert game.move(1, 2, 2) == 0
assert game.move(2, 2, 1) == 1  # Player 1 wins diagonally




# ## 这个是简单的tictactoe n个子胜利，nxn的棋盘
# class TicTacToe:

#     def __init__(self, n: int):
#         self.rows = [0]*n
#         self.cols = [0]*n
#         self.diag = 0
#         self.antidiag = 0
#         self.n = n
        

#     def move(self, row: int, col: int, player: int) -> int:
#         n = self.n
#         move = 1 if player==1 else -1
#         self.rows[row] += move
#         self.cols[col] += move
#         if row == col:
#             self.diag += move
#         if row+col == n-1:
#             self.antidiag += move
        
#         if self.rows[row] == n*move or self.cols[col] == n*move or self.diag == n*move or self.antidiag == n*move:
#             return player
#         return 0

        


# Your TicTacToe object will be instantiated and called as such:
# obj = TicTacToe(n)
# param_1 = obj.move(row,col,player)

# LC 695 Max Area of Island

In [4]:
def maxArea(grid):
    res = 0
    ROWS, COLS = len(grid), len(grid[0])

    def dfs(r, c):
        if (r<0 or c<0 or r>=ROWS or c>=COLS) or grid[r][c] != 1:
            return 0
        
        grid[r][c] = -1

        return 1 + dfs(r-1, c) + dfs(r+1, c) + dfs(r, c+1) + dfs(r, c-1)

    for r in range(ROWS):
        for c in range(COLS):
            if grid[r][c] == 1:
                res = max(res, dfs(r, c))
    return res

        
grid = [[0,0,1,0,0,0,0,1,0,0,0,0,0],[0,0,0,0,0,0,0,1,1,1,0,0,0],[0,1,1,0,1,0,0,0,0,0,0,0,0],[0,1,0,0,1,1,0,0,1,0,1,0,0],[0,1,0,0,1,1,0,0,1,1,1,0,0],[0,0,0,0,0,0,0,0,0,0,1,0,0],[0,0,0,0,0,0,0,1,1,1,0,0,0],[0,0,0,0,0,0,0,1,1,0,0,0,0]]
maxArea(grid) # expect to return 6


6

# LC787，有点像通勤题目

In [39]:
from collections import defaultdict, deque
from heapq import heappush, heappop

def findCheapestPrice(self, n: int, flights, src: int, dst: int, k: int) -> int:
    # BFS
    """
    1. construct a graph, adjacency list
    2. level by level expansion with at most k+1 depth
    3. at every level, we try to update to the min cost neighbor

    time: 
    n cities
    e flights
    k
    O(n+ek)
    space:
    O(ek+n), q is e, graph is e, costs is n.
    """


    graph = defaultdict(list)
    for s, d, price in flights:
        graph[s].append((d, price))
    
    q = deque()
    q.append((src, 0)) # node, cost
    costs = [float('inf')] * n

    while q and k>=0:
        sz = len(q)
        for _ in range(sz):
            node, cost = q.popleft()
            for neighbor, price in graph[node]:
                if cost + price < costs[neighbor]:
                    costs[neighbor] = cost + price
                    q.append((neighbor, costs[neighbor]))
        k -= 1
    return costs[dst] if costs[dst] != float('inf') else -1

def findCheapestPrice(self, n: int, flights, src: int, dst: int, k: int) -> int:
    """
    Dijkstra (priority q + at most k stops)

    1. use minheap, based on curr min cost to expand cities
    2. record (cost, node, stops), when stops>k, we can prune
    3. return res when reaching dest

    time: O(n+eklog(ek))
    space: O(n+ek)
    """
    graph = defaultdict(list)
    for s, e, p in flights:
        graph[s].append((e, p)) # start city: (end city, price), time: O(n)

    visited = {} # (city, stops) -> cost

    heap = [(0, src, 0)] # (cost, city, stops)
    
    while heap: # max size of heap is ek
        cost, city, stops = heappop(heap)
        if city == dst:
            return cost
        
        if (city, stops) in visited and visited[(city, stops)] <= cost:
            continue

        visited[(city, stops)] = cost
        

        if stops <= k:
            for neighbor, price in graph[city]:
                heappush(heap, (cost+price, neighbor, stops+1))
    
    return -1



# Encoding Decoding

In [2]:
# Run-Length Encoding (RLE) : Encoding scheme where integers that have the same value can be represented as a single value with a count, e.g. 1, 1, 1, 1, 1 can be written as 1 (5).

# Bit-Packing (BP) : Encoding scheme where smaller integer values can be represented without using the full 4 bytes of data. For example, all numbers in 2, 3, 4, 5 fit within 3 bits, so we can encode ("pack") them into 12 bits or at most 2 bytes thus reducing data 8x.

# Run is a sequence of values that are encoded using RLE or BP. All values in a run have the same encoding but runs themselves can interleave, e.g. [RLE, BP, RLE, RLE, BP, ...].

#  1. All values have to be placed into runs, and value order cannot change.
#  2. A Run-Length encoded run has at least 8 values (only the last run can have
#     fewer values).
#  3. Once started, a Run-Length encoded run should extend as far as possible
#     (it should continue as long as the input values are repeating).
#  4. A Bit-Packing encoded run has exactly 8 values, i.e. once it is started,
#     it is required to fill all 8 values (only the last run can have fewer values).
#  5. Run-Length encoding is our default scheme.  We only use Bit-Packing encoding
#     when we cannot apply Run-Length (because there is not enough repeated values).


# [1, 1, 1] will be encoded as RLE[1, 3].
# [1, 1, 1, 1, 2, 3, 4, 5] will be encoded as BP[1, 1, 1, 1, 2, 3, 4, 5].
# [1, 1, 1, 1, 1, 1, 1, 1, 2, 3, 4, 5] will be encoded as RLE[1, 8], BP[2, 3, 4, 5].
# [1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 3, 4, 5] will be encoded as RLE[1, 9], BP[2, 3, 4, 5].


# This file contains all of the necessary code for the exercise.
#
# Your goal is to implement Encoder and Decoder specification by combining
# RLE and BP encodings.
#
# You should implement all methods containing `# TODO` comment.
# You may use the API helper methods and classes provided below.

In [43]:
from typing import List, Union

class Run:
    def __init__(self, kind: str, values: List[int]):
        self.kind = kind  # "RLE" or "BP"
        self.values = values

    def __repr__(self):
        return f"{self.kind}[{', '.join(map(str, self.values))}]"

class Encoder:
    def encode(self, input: List[int]) -> List[Run]:
        i = 0
        n = len(input)
        result = []

        while i < n:
            # Try RLE run
            val = input[i]
            j = i
            while j < n and input[j] == val:
                j += 1
            count = j - i

            if count >= 8:
                # RLE run
                result.append(Run("RLE", [val, count]))
                i = j
            else:
                # Try BP run of exactly 8 values if possible
                bp_run = []
                while i < n and len(bp_run) < 8:
                    val = input[i]
                    # Try to extend RLE instead
                    if len(bp_run) >= 8:
                        break

                    if i + 7 < n and all(input[i + k] == input[i] for k in range(8)):
                        break  # start of a valid RLE, prefer RLE
                    bp_run.append(val)
                    i += 1
                result.append(Run("BP", bp_run))
        return result

class Decoder:
    def decode(self, runs: List[Run]) -> List[int]:
        output = []
        for run in runs:
            if run.kind == "RLE":
                val, count = run.values
                output.extend([val] * count)
            elif run.kind == "BP":
                output.extend(run.values)
        return output

# --- Example test ---
if __name__ == '__main__':
    encoder = Encoder()
    decoder = Decoder()

    test = [1, 1, 1, 1, 1, 1, 1, 1, 2, 3, 4, 5]
    runs = encoder.encode(test)
    print("Encoded:", runs)
    output = decoder.decode(runs)
    print("Decoded:", output)
    assert output == test

Encoded: [RLE[1, 8], BP[2, 3, 4, 5]]
Decoded: [1, 1, 1, 1, 1, 1, 1, 1, 2, 3, 4, 5]


# Lazy Array
problem

上来就给了API，然后让我猜Api什么样 的behavior才更符合用户的期待。
lazyarray, 写出来了+test case test了, 注意需要test lazyness，用wrapper或者mock check function invocation times, 这个follow up写完没时间测试了，估计挂在了这里。


In [44]:
from typing import Callable, List

class LazyArray:
    def __init__(self, data: List):
        self.data = data
        self.transforms: List[Callable] = []

    def map(self, func: Callable):
        self.transforms.append(func)
        return self  # 支持链式调用

    def get(self, index: int):
        value = self.data[index]
        for func in self.transforms:
            value = func(value)
        return value

    def __len__(self):
        return len(self.data)

    def materialize(self):
        # Optional: 将所有变换应用并返回新数组
        return [self.get(i) for i in range(len(self))]
    
arr = LazyArray([1, 2, 3])
arr = arr.map(lambda x: x + 1).map(lambda x: x * 10)

print(arr.get(0))  # (1+1)*10 = 20
print(arr.get(1))  # (2+1)*10 = 30
print(arr.get(2))  # (3+1)*10 = 40

20
30
40


# LC198 House Robber
follow up 1: 213 - House Robber II
follow up 2: 改变间隔k个

In [ ]:
nums = [2,7,9,3,1]
"""
can't rob in neighboring house

dp[i]: max revenue at house i

dp[0] = 0
dp[1] = nums[0]
dp[2] = max(nums[0], nums[1])

dp[i] = rob i vs not rob i
rob i: nums[i] + dp[i-2]
not rob i: dp[i-1]

"""

def rob(nums):
    n = len(nums)
    dp = [0] * (n)
    
    if n == 1: return nums[0]

    dp[0] = nums[0]
    dp[1] = max(dp[0], nums[1])

    for i in range(2, n):
        dp[i] = max(nums[i]+dp[i-2], dp[i-1])
    
    return dp[-1]

def rob2(nums):
    # split into two questions
    # we will not rob 1st house
    # we will not rob last house
    
    return max(rob(nums[:-1]), rob(nums[1:]))

def rob3(nums, k):
    n = len(nums)
    if n == 0:
        return 0
    dp = [0] * n
    dp[0] = nums[0]
    for i in range(1, n):
        # 如果不rob house i
        prev = dp[i-1]
        # 如果rob house i，就要从i-k-1的地方跳过来，如果越界，就是从0跳过来
        jump = nums[i] + (dp[i-k-1] if i-k-1>=0 else 0)
        dp[i] = max(prev, jump)
    return dp[-1]



nums = [2,7,9,3,1]
k = 2

print(f"rob v1 is {rob(nums)}")
print(f"rob v2 is {rob2(nums)}")
print(f"rob v3 is {rob3(nums, k)}")

rob v1 is 12
rob v2 is 11
rob v3 is 9


# Deisgn Hit Counter LC 362
Circular buffer： Circular Buffer（循环缓冲区）是一个固定大小的数组结构，可以像环一样“首尾相接”，常用于实现滑动窗口统计、实时数据流处理、队列缓存等高性能应用。




电话面试的题目；
用deque 不是特别efficient，如果有million of hits per seconds, the memory usage will explode. O(n)

Better approach, use bucketed aggregation. Keep a fixed-array of 300 buckets (one per second in the 5-min window)
each bucket stores 
1. the latest timestamp it was updated
2. the number of hits at that timestamp


# 下面的题目考了也认了，不适合我

# Circuit/Double breaker
一道模拟 circuit breaker 的题，大致就是有一个primary server 和一个secondary server， 你要去模拟一个circuit breaker， 比如primary fail 就retry secondary。 然后某个server 连续fail了几次就直接拒绝，连续拒绝几次就重新可以retry。总之题目描述有点长，但是根据描述细心点还是不难实现， 但是可以想想怎么让逻辑简单清晰一点。

简单来说就是需要为我们的server写一个类似getaway处理所有request，当server连续return多次failure之后， gateway不会试图继续request server，而是直接return rejection。多次连续rejection之后再去尝试request server。然后假设我们有一个primary和一个backup两个server，需要写另一个interface先去试primary然后试backup。关键是这个题最后要写成类似mock的形式，题目也很复杂看得头晕，问了一大堆clarification，最后勉强跑完了一个test case。建议各位好好看一下自己语言lambda function或者function object的syntax。
https://netflixtechblog.com/making-the-netflix-api-more-resilient-a8ec62159c2d

In [45]:
import time
import random
from enum import Enum

# 定义断路器状态
class CircuitState(Enum):
    CLOSED = 1
    OPEN = 2
    HALF_OPEN = 3

class CircuitBreaker:
    """
    实现一个简单的断路器模式。
    """
    def __init__(self, name: str, failure_threshold: int, rejection_timeout_ms: int, half_open_success_threshold: int):
        """
        初始化断路器。
        Args:
            name (str): 断路器的名称（例如 "Primary CB", "Secondary CB"）。
            failure_threshold (int): 在 CLOSED 状态下，连续失败多少次后断路器会跳闸到 OPEN 状态。
            rejection_timeout_ms (int): 在 OPEN 状态下，持续多长时间（毫秒）后断路器会尝试进入 HALF_OPEN 状态。
            half_open_success_threshold (int): 在 HALF_OPEN 状态下，连续成功多少次后断路器会恢复到 CLOSED 状态。
        """
        self.name = name
        self.failure_threshold = failure_threshold
        self.rejection_timeout_ms = rejection_timeout_ms
        self.half_open_success_threshold = half_open_success_threshold

        self.current_state = CircuitState.CLOSED
        self.failure_count = 0        # CLOSED 状态下连续失败次数
        self.success_count = 0        # HALF_OPEN 状态下连续成功次数
        self.last_trip_time = 0       # 进入 OPEN 状态的时间戳（毫秒）

        print(f"[{self.name}] 断路器初始化，状态: {self.current_state.name}")

    def allow_request(self) -> bool:
        """
        判断是否允许请求通过断路器。
        Returns:
            bool: 如果允许请求则返回 True，否则返回 False。
        """
        if self.current_state == CircuitState.CLOSED:
            return True
        elif self.current_state == CircuitState.OPEN:
            current_time_ms = int(time.time() * 1000)
            # 如果处于 OPEN 状态的时间超过了设定的超时时间，则尝试进入 HALF_OPEN
            if current_time_ms - self.last_trip_time > self.rejection_timeout_ms:
                self._transition_to_half_open()
                return True # 进入 HALF_OPEN 后，允许一个请求通过来探测服务是否恢复
            else:
                return False # 仍在 OPEN 状态，直接拒绝请求
        elif self.current_state == CircuitState.HALF_OPEN:
            # HALF_OPEN 状态允许请求通过，用于探测服务是否恢复
            # （这里简化为每次只允许一个请求，更复杂的实现可能允许少量并发请求）
            return True 
        return False # 不应该发生

    def record_success(self):
        """
        记录一次成功调用。
        """
        if self.current_state == CircuitState.CLOSED:
            self.failure_count = 0 # 成功一次，重置连续失败计数
        elif self.current_state == CircuitState.HALF_OPEN:
            self.success_count += 1
            # 如果 HALF_OPEN 状态下连续成功次数达到阈值，则恢复到 CLOSED
            if self.success_count >= self.half_open_success_threshold:
                self._transition_to_closed()
        # print(f"[{self.name}] 记录成功。状态: {self.current_state.name}, 连续失败: {self.failure_count}, 连续成功: {self.success_count}")

    def record_failure(self):
        """
        记录一次失败调用。
        """
        if self.current_state == CircuitState.CLOSED:
            self.failure_count += 1
            # 如果 CLOSED 状态下连续失败次数达到阈值，则跳闸到 OPEN
            if self.failure_count >= self.failure_threshold:
                self._transition_to_open()
        elif self.current_state == CircuitState.HALF_OPEN:
            # 如果 HALF_OPEN 状态下失败，立即返回 OPEN 状态
            self._transition_to_open() 
        # print(f"[{self.name}] 记录失败。状态: {self.current_state.name}, 连续失败: {self.failure_count}, 连续成功: {self.success_count}")

    def _transition_to_open(self):
        """
        内部方法：状态切换到 OPEN。
        """
        self.current_state = CircuitState.OPEN
        self.last_trip_time = int(time.time() * 1000) # 记录进入 OPEN 的时间
        self.failure_count = 0 # 重置失败计数
        self.success_count = 0 # 重置成功计数
        print(f"[{self.name}] 状态切换到 OPEN，跳闸时间: {self.last_trip_time}")

    def _transition_to_half_open(self):
        """
        内部方法：状态切换到 HALF_OPEN。
        """
        self.current_state = CircuitState.HALF_OPEN
        self.success_count = 0 # 重置成功计数，开始探测
        print(f"[{self.name}] 状态切换到 HALF_OPEN")

    def _transition_to_closed(self):
        """
        内部方法：状态切换到 CLOSED。
        """
        self.current_state = CircuitState.CLOSED
        self.failure_count = 0 # 重置失败计数
        self.success_count = 0 # 重置成功计数
        print(f"[{self.name}] 状态切换到 CLOSED")

# IP CIDR
真的不想看。。。
给一对CIDR和一个ip address，判断ip是否会被rules allow

followup：给一个CIDR，看看能否被firewall rules allow
List of rules determine which IP could pass, which IP will be denied
https://leetcode.com/discuss/post/4007625/databricks-oa-software-engineer-l4-by-an-0nb5/
https://www.1point3acres.com/bbs/thread-1075625-1-1.html

In [46]:
import ipaddress
from typing import List, Dict, Union

# --- 工具函数 ---

def ip_to_int(ip_str: str) -> int:
    """将 IP 地址字符串转换为 32 位整数。"""
    return int(ipaddress.IPv4Address(ip_str))

def cidr_to_network_object(cidr_str: str) -> ipaddress.IPv4Network:
    """将 CIDR 字符串转换为 ipaddress.IPv4Network 对象。"""
    # strict=False 允许主机位非零的 CIDR 字符串 (如 192.168.1.1/24) 自动转换为网络地址 (192.168.1.0/24)
    return ipaddress.IPv4Network(cidr_str, strict=False)

# --- 主函数：判断单个 IP 是否被允许 ---

def is_ip_allowed(ip_address: str, firewall_rules: List[Dict[str, str]]) -> bool:
    """
    判断一个给定的 IP 地址是否被防火墙规则允许。
    规则列表按顺序处理，第一个匹配的规则决定结果。

    Args:
        ip_address (str): 要检查的 IP 地址字符串。
        firewall_rules (List[Dict[str, str]]): 防火墙规则列表。
                                                每条规则是一个字典，包含 "cidr" 和 "action"。
                                                例如: {"cidr": "192.168.1.0/24", "action": "allow"}

    Returns:
        bool: 如果 IP 被允许则返回 True，否则返回 False。
    """
    ip_obj = ipaddress.IPv4Address(ip_address)

    for rule in firewall_rules:
        rule_cidr_str = rule["cidr"]
        rule_action = rule["action"]
        
        rule_network_obj = cidr_to_network_object(rule_cidr_str)

        # 判断 IP 是否在当前规则的 CIDR 范围内
        if ip_obj in rule_network_obj:
            print(f"  IP {ip_address} 匹配到规则: CIDR {rule_cidr_str}, Action {rule_action}")
            return rule_action == "allow"
    
    # 如果遍历完所有规则都没有匹配到，默认拒绝
    print(f"  IP {ip_address} 未匹配任何规则，默认拒绝。")
    return False

# --- Follow-up：判断一个 CIDR 是否被允许 ---

def is_cidr_allowed(target_cidr_str: str, firewall_rules: List[Dict[str, str]]) -> bool:
    """
    判断一个给定的目标 CIDR 是否被防火墙规则允许。
    这里采用的策略是：
    一个目标 CIDR 被允许，当且仅当：
    1. 它没有被任何一条 'deny' 规则明确拒绝。
    2. 并且它被至少一条 'allow' 规则明确允许。
    
    这是一种常见的面试简化，其核心在于确保目标 CIDR 不会因任何拒绝规则而被“污染”。
    如果目标 CIDR与任何拒绝规则有交集，则认为其不被完全允许。
    如果没有任何拒绝规则，则再看是否有允许规则覆盖它。

    Args:
        target_cidr_str (str): 要检查的目标 CIDR 字符串。
        firewall_rules (List[Dict[str, str]]): 防火墙规则列表。

    Returns:
        bool: 如果目标 CIDR 被认为完全允许则返回 True，否则返回 False。
    """
    target_network_obj = cidr_to_network_object(target_cidr_str)

    # 1. 检查是否存在任何 'deny' 规则与目标 CIDR 有交集
    # 如果有任何 deny 规则与目标 CIDR 有交集，那么目标 CIDR 中至少有一部分会被拒绝。
    # 根据题意“判断IP是否会被rules allow”，通常意味着这个IP必须被允许，而不是被拒绝。
    # 对于CIDR，如果其任何部分被拒绝，那么整个CIDR通常就不能被“允许”。
    # 这里我们简化处理：如果任何 deny 规则与 target_cidr 有交集，则认为不被允许。
    # （更严格的防火墙会按顺序和最长匹配来决定）
    
    # 先构建一个拒绝规则的集合，便于后续查找
    deny_rules = [rule for rule in firewall_rules if rule["action"] == "deny"]
    
    for rule in deny_rules:
        rule_network_obj = cidr_to_network_object(rule["cidr"])
        if target_network_obj.overlaps(rule_network_obj):
            print(f"  目标 CIDR {target_cidr_str} 与拒绝规则 {rule['cidr']} 有交集。因此不被允许。")
            return False

    # 2. 如果没有任何 'deny' 规则与目标 CIDR 有交集，
    # 那么我们接下来检查是否有 'allow' 规则能够完全包含目标 CIDR。
    # （这里假设如果没有任何 deny 规则，只要有 allow 规则覆盖即可，
    # 即使 allow 规则不完全覆盖目标 CIDR 外部的部分，
    # 只要它完全包含了目标 CIDR，我们就算它被允许。）
    
    # 寻找一个能够完全包含目标 CIDR 的 "allow" 规则
    found_allow_covering_target = False
    for rule in firewall_rules:
        if rule["action"] == "allow":
            rule_network_obj = cidr_to_network_object(rule["cidr"])
            # 检查规则的 CIDR 是否完全包含目标 CIDR
            if target_network_obj.subnet_of(rule_network_obj):
                print(f"  目标 CIDR {target_cidr_str} 被允许规则 {rule['cidr']} 完全包含。")
                found_allow_covering_target = True
                break # 找到一个即可
    
    # 如果找到一个完全包含目标 CIDR 的 allow 规则，且之前没有被 deny 规则污染
    if found_allow_covering_target:
        return True
    
    # 如果没有任何拒绝规则，也没有允许规则覆盖目标 CIDR，则根据默认行为（通常是拒绝）
    print(f"  目标 CIDR {target_cidr_str} 未被任何拒绝规则交叉，但也没有被允许规则完全包含。默认拒绝。")
    return False